![NVIDIA Logo](images/nvidia.png)

## Imports

In [1]:
import json
import random
import pandas as pd

from tqdm.notebook import tqdm

from llm_utils.nemo_service_models import NemoServiceBaseModel
from llm_utils.models import PubmedModels
from llm_utils.helpers import plot_experiment_results, accuracy_score
from llm_utils.pubmedqa import generate_prompt_and_answer, strip_response
from llm_utils.prompt_creators import create_prompt_with_examples, create_nemo_prompt_with_examples

## List Models

In [2]:
PubmedModels.list_models()

gpt8b: gpt-8b-000
gpt20b: gpt20b
gpt43b: gpt-43b-001


### Non-Instruction Fine-tuned Prompt Format

GPT20B and GPT8B are not instruction fine-tuned models, and thus we only need to provide our example prompt/response shots in a straightforward manner. We've provided the `create_prompt_with_examples` helper function to accomplish this.

In [3]:
llms = {}

In [4]:
llms['gpt8b'] = NemoServiceBaseModel(PubmedModels.gpt8b.value, create_prompt_with_examples=create_prompt_with_examples)
llms['gpt20b'] = NemoServiceBaseModel(PubmedModels.gpt20b.value, create_prompt_with_examples=create_prompt_with_examples)
llms['gpt43b'] = NemoServiceBaseModel(PubmedModels.gpt43b.value, create_prompt_with_examples=create_nemo_prompt_with_examples)

In [5]:
llms

{'gpt8b': <llm_utils.nemo_service_models.NemoServiceBaseModel at 0x7f04567d1310>,
 'gpt20b': <llm_utils.nemo_service_models.NemoServiceBaseModel at 0x7f04567d16d0>,
 'gpt43b': <llm_utils.nemo_service_models.NemoServiceBaseModel at 0x7f04567d12e0>}

### One shot 

In [6]:
!pwd

/workspace/dli/2-PubMedQA


In [8]:
df = pd.read_csv('/workspace/dli/2-PubMedQA/data_62124.csv')

In [9]:
df.head()

,Unnamed: 0,good address,compare address,label
0,0,14361 cupcake terrace,14361 cupofcake terrace,1
1,1,13823 principle terrace,2689 principal highway,0
2,2,908 toolbox street,1209 main highway,0
3,3,6785 wheel avenue,1809 main way,0
4,4,3944 electric screwdriver road,3944 elecc screwdriver,1


### With simple prompting 

In [15]:
rowNum = 10 
c = 0
for i in range(rowNum):
    #prompt = f'Should the following two addresses be linked {df['good address'][i]} and {df['compare address'][i]}?'
    prompt = f'Are the following addresses mispelled versions of the same address or two distinct addresses: {df['good address'][i]} and {df['compare address'][i]}? ANSWER (yes|no)'
    
    response = llms['gpt20b'].generate(prompt, tokens_to_generate=1, return_type='text').strip()

    if df['label'][i] == 1:
        label = 'Yes'
    else:
        label = 'No'

    print(f'Good Address: {df['good address'][i]}')
    print(f'Compare Address: {df['compare address'][i]}')
    
    print(f'Response from model: {response}')
    print(f'Actual answer: {label}')
    correct = label == response
    print(f'Response from model correct: {correct}\n')
    if correct:
        c += 1

print(f'the accuracy is {c/rowNum}')


Good Address: 14361 cupcake terrace
Compare Address: 14361 cupofcake terrace
Response from model: 
Actual answer: Yes
Response from model correct: False

Good Address: 13823 principle terrace
Compare Address: 2689 principal highway
Response from model: 
Actual answer: No
Response from model correct: False

Good Address: 908 toolbox street
Compare Address: 1209 main highway
Response from model: 
Actual answer: No
Response from model correct: False

Good Address: 6785 wheel avenue
Compare Address: 1809 main way
Response from model: 
Actual answer: No
Response from model correct: False

Good Address: 3944 electric screwdriver road
Compare Address: 3944 elecc screwdriver
Response from model: 
Actual answer: Yes
Response from model correct: False

Good Address: 1791 burdens square
Compare Address: 3228 burden place
Response from model: 
Actual answer: No
Response from model correct: False

Good Address: 10569 tablet highway
Compare Address: 10569 tmablevt highway
Response from model: 
Actua

### Formatted prompting

In [14]:
rowNum = 10 
c = 0
for i in range(rowNum):
    prompt = f'''Objective: Some addresses look the same, but indicate different physical locations and 
    therefore they should not be linked, one indication that two addresses should not be linked is they have different
    house numbers. Alternatively, some addresses are not exact matches but should be linked, they often dont match because
    of a mispelling in one of the street names.  For example, These pair of addresses should be matched: 990 sizzling place and 990 sizlig place. while, 
    this pair of streets 67 metal way and 87 petal drive indicate different address that should not be matched. \nQUESTION:
    Should the following two addresses be linked {df['good address'][i]} and {df['compare address'][i]}? \nANSWER (yes|no):'''
    
    
    response = llms['gpt20b'].generate(prompt, tokens_to_generate=1, return_type='text').strip()

    if df['label'][i] == 1:
        label = 'yes'
    else:
        label = 'no'

    print(f'Good Address: {df['good address'][i]}')
    print(f'Compare Address: {df['compare address'][i]}')
    
    print(f'Response from model: {response}')
    print(f'Actual answer: {label}')
    correct = label == response
    print(f'Response from model correct: {correct}\n')
    if correct:
        c += 1

print(f'the accuracy is {c/rowNum}')

Good Address: 14361 cupcake terrace
Compare Address: 14361 cupofcake terrace
Response from model: yes
Actual answer: yes
Response from model correct: True

Good Address: 13823 principle terrace
Compare Address: 2689 principal highway
Response from model: yes
Actual answer: no
Response from model correct: False

Good Address: 908 toolbox street
Compare Address: 1209 main highway
Response from model: yes
Actual answer: no
Response from model correct: False

Good Address: 6785 wheel avenue
Compare Address: 1809 main way
Response from model: yes
Actual answer: no
Response from model correct: False

Good Address: 3944 electric screwdriver road
Compare Address: 3944 elecc screwdriver
Response from model: yes
Actual answer: yes
Response from model correct: True

Good Address: 1791 burdens square
Compare Address: 3228 burden place
Response from model: yes
Actual answer: no
Response from model correct: False

Good Address: 10569 tablet highway
Compare Address: 10569 tmablevt highway
Response fr

### Formatting/43B Model/ One Shot

In [13]:
rowNum = 10 
c = 0
for i in range(rowNum):
    prompt = f'''Objective: Some addresses look the same, but indicate different physical locations and 
    therefore they should not be linked, one indication that two addresses should not be linked is they have different
    house numbers. Alternatively, some addresses are not exact matches but should be linked, they often dont match because
    of a mispelling in one of the street names.  For example, These pair of addresses should be matched: 990 sizzling place and 990 sizlig place. while, 
    this pair of streets 67 metal way and 87 petal drive indicate different address that should not be matched. \nQUESTION:
    Should the following two addresses be linked {df['good address'][i]} and {df['compare address'][i]}? \nANSWER (yes|no):'''
    
    
    response = llms['gpt43b'].generate(prompt, tokens_to_generate=1, return_type='text').strip()

    if df['label'][i] == 1:
        label = 'Yes'
    else:
        label = 'no'

    print(f'Good Address: {df['good address'][i]}')
    print(f'Compare Address: {df['compare address'][i]}')
    
    print(f'Response from model: {response}')
    print(f'Actual answer: {label}')
    correct = label == response
    print(f'Response from model correct: {correct}\n')
    if correct:
        c += 1

print(f'the accuracy is {c/rowNum}')

Good Address: 14361 cupcake terrace
Compare Address: 14361 cupofcake terrace
Response from model: no
Actual answer: Yes
Response from model correct: False

Good Address: 13823 principle terrace
Compare Address: 2689 principal highway
Response from model: no
Actual answer: no
Response from model correct: True

Good Address: 908 toolbox street
Compare Address: 1209 main highway
Response from model: no
Actual answer: no
Response from model correct: True

Good Address: 6785 wheel avenue
Compare Address: 1809 main way
Response from model: no
Actual answer: no
Response from model correct: True

Good Address: 3944 electric screwdriver road
Compare Address: 3944 elecc screwdriver
Response from model: no
Actual answer: Yes
Response from model correct: False

Good Address: 1791 burdens square
Compare Address: 3228 burden place
Response from model: no
Actual answer: no
Response from model correct: True

Good Address: 10569 tablet highway
Compare Address: 10569 tmablevt highway
Response from model